In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.style.use('ggplot')

df = pd.read_csv(....)
print(df.head())

## Cleansing

In [ ]:
df.info()
df.describe()

- karena Purchase date bertipe object maka kita rubah menjadi date
- untuk return terdapat null lalu kita isi dengan 0 rubah juga tipe data nya menjadi int

In [ ]:
# merubah Purchase Date jadi tipe date
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# mengisi data yang null di return dengan 0 lalu rubah tipe nya ke int
df['Returns'] = df['Returns'].fillna(0).astype(int)

# hapus kolom customer age
df = df.drop(columns=['Customer Age'])

# filter data dari bulan september 2023
df = df[df['Purchase Date'] < '2023-09-01']

In [ ]:
# membuat kolom pendapatan bersih
df['Net Revenue'] = df['Total Purchase Amount'] * (1 - df['Returns'])

# ekstras waktu

df['Year'] = df['Purchase Date'].dt.year
df['Month_Year'] = df['Purchase Date'].dt.to_period('M')

#mengelompokan usia

def categorize_age(age):
    if age < 20: return 'Teenager'
    elif age < 30: return 'Young adult'
    elif age < 50: return 'Adult'
    else: return 'Senior'

df['Age Group'] = df['Age'].apply(categorize_age)

In [ ]:
plt.Figure(figsize=(15, 5))

# distribusi kelompok usia
plt.subplot(1,2,1)
sns.countplot(data=df, x='Age Group', order=['Teenager', 'Young Adult', 'Adult', 'Senior'], palette='viridis')
plt.title('Distribusi Kelompok Usia Pelanggan')

# Proporsi metode pembayaran
plt.subplot(1,2,2)
df['Payment Method'].value_counts().plot.pie(autopct='%1.1f%%', colors=sns.color_palette('pastel'), startangle=90)
plt.title('Metode Pembayaran Paling Populer')

plt.tight_layout()
plt.show()

In [ ]:
# 1. Tren Pendapatan Bulanan
plt.figure(figsize=(15, 6))
monthly_sales = df.groupby('Month_Year')['Net Revenue'].sum()
monthly_sales.plot(kind='line', marker='o', color='teal', linewidth=2)
plt.title('Tren Pendapatan Bersih Bulanan (2020 - 2023)')
plt.xticks(rotation=45)
plt.show()

# 2. Kategori Produk Berdasarkan Kelompok Usia
plt.figure(figsize=(12, 6))
sns.countplot(data=df, x='Age Group', hue='Product Category', order=['Teenager', 'Young Adult', 'Adult', 'Senior'])
plt.title('Preferensi Kategori Produk per Kelompok Usia')
plt.legend(title='Kategori', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
# 1. Heatmap Korelasi Numerik
plt.figure(figsize=(10, 8))
numeric_df = df.select_dtypes(include=[np.number])
sns.heatmap(numeric_df.corr(), annot=True, cmap='RdBu_r', center=0)
plt.title('Matriks Korelasi Variabel')
plt.show()

# 2. Cek Outlier
plt.figure(figsize=(12, 5))
sns.boxplot(data=df[['Product Price', 'Total Purchase Amount']], palette='Set3')
plt.title('Pengecekan Outlier pada Harga & Total Belanja')
plt.show()

In [ ]:
import os
os.makedirs('../data/processed', exist_ok=True)

# Simpan hasil akhir yang sudah sangat bersih
df.to_csv('../data/processed/ecommerce_cleaned.csv', index=False)
print("✅ SEMUA PROSES EDA SELESAI!")
print("Data siap digunakan untuk modeling di: data/processed/ecommerce_cleaned.csv")